# RO47002 Machine Learning for Robotics
* (c) TU Delft, 2020
* Period: 2020-2021, Q1
* Course homepage: https://brightspace.tudelft.nl/d2l/home/318952

Make sure you fill in any place that says `YOUR CODE HERE` or `YOUR ANSWER HERE`. Moreover, if you see an empty cell, please DO NOT delete it, instead run that cell as you would run all other cells. Please fill in your name(s) and other required details below:

In [5]:
GROUP_NUMBER = "8"
STUDENT_NAME1 = "Rishabh Singh Jakhar"
STUDENT_NUMBER1 = "4779819"
STUDENT_NAME2 = "Shantanu Shivankar"
STUDENT_NUMBER2 = "5032814"

In [6]:
# Note: this block is a check that you have filled in the above information.
# It will throw an AssertionError until all fields are filled
assert(GROUP_NUMBER != "")
assert(STUDENT_NAME1 != "")
assert(STUDENT_NUMBER1 != "")
assert(STUDENT_NAME2 != "")
assert(STUDENT_NUMBER2 != "")

### General announcements

* Do *not* share your solutions, and do *not* copy solutions from others. By submitting your solutions, you claim that you and your lab partner alone are responsible for this code.

* Do *not* email questions directly, since we want to provide everybody with the same information and avoid repeating the same answers. Instead, please post your questions regarding this assignment in the correct support forum on Brightspace, this way everybody can benefit from the response. If you do have a particular question that you want to ask directly, please use the scheduled practicum hours to ask a TA.

* There is a strict deadline for each assignment. Students are responsible to ensure that they have uploaded their work in time. So, please double check that your upload succeeded to the Brightspace and avoid any late penalties.

* This [Jupyter notebook](https://jupyter.org/) uses `nbgrader` to help us with automated tests. `nbgrader` will make various cells in this notebook "uneditable" or "unremovable" and gives them a special id in the cell metadata. This way, when we run our checks, the system will check the existence of the cell ids and verify the number of points and which checks must be run. While there are ways that you can edit the metadata and work around the restrictions to delete or modify these special cells, you should not do that since then our nbgrader backend will not be able to parse your notebook and give you points for the assignment. You are free to add additional cells, but if you find a cell that you cannot modify or remove, please know that this is on purpose.

* This notebook will have in various places a line that throws a `NotImplementedError` exception. These are locations where the assignment requires you to adapt the code! These lines are just there as a reminder for youthat you have not yet adapted that particular piece of code, especially when you execute all the cells. Once your solution code replaced these lines, it should accordingly *not* throw any exceptions anymore.

Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

---

# Practicum 4
* Before performing this practicum, work through Book chapters: 5, 6, 7
* **Deadline**: Sunday, October 4, 2020, 23:59

## Objectives
### Part 1
* Discover a data set and prepare it for algorithms
* Inspect a decision tree
* Re-implement a linear SVM
* Re-implement a voting classifier
* Explore how bagging improves a very simple classifier

### Part 2 (Imitation Learning and Motion Planning)
* Build a dataset using an oracle
* Use a classifier to imitate the behavior of a motion planning algorithm
* Explore how the construction of the dataset can affect the performance of the classifier


### Setup common python stuff
We will start by loading a few common python dependencies

In [7]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ == "0.23.2"

# Common imports
import numpy as np
import pandas as pd
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

In [8]:
def plot_feature_space_function(f, X, y, axes, class_1=0, class_2=1, res=0.1):
    """ Plot the 2D feature space of the first two features of the data in X.
        For the feature space, plot the samples in X with their class labels y,
        and also overlay a countour plot with the function f(x) evaluated at a grid within
        the shown region of the feature space (the size of the region is determined by the extent of the data).
        based on https://scikit-learn.org/stable/auto_examples/ensemble/plot_voting_decision_regions.html
    """
    
    if f is not None:
        xx, yy = np.meshgrid(np.arange(axes[0], axes[1], res),np.arange(axes[2], axes[3], res))

        Z = f(np.c_[xx.ravel(), yy.ravel()])
        Z = Z.reshape(xx.shape)

        plt.contourf(xx, yy, Z, alpha=0.4)
        
    plt.plot(X[y==class_1,0], X[y==class_1,1], '.', label='class '+str(class_1), zorder=-1)
    plt.plot(X[y==class_2,0], X[y==class_2,1], '.', label='class '+str(class_2), zorder=-1)
    plt.grid('on')
    plt.axis(axes)
    plt.legend()

In [9]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

def print_accuracy_confusion(y_test, y_pred, clf_name=''):
    print(clf_name, 'accuracy score:', accuracy_score(y_test, y_pred))
    print(clf_name, 'confusion matrix:\n', confusion_matrix(y_test, y_pred))

## Exploring and preparing a dataset
We are going to use a dataset for classifying hand gestures from EMG signals for the first part of this practicum. The description and original source can be found here: https://www.kaggle.com/kyr7plus/emg-4

This kind of data can not only used for controlling prosthetics, but also for other human-robot interaction applications. Here is a little example from another CoR stuff member, Luka Peternel, on the use of EMG sensors in human-robot interaction https://youtu.be/i42X_rVVQWU

Your first task is to read the description on the website and to figure out how the dataset is structured.
Load the correct file (included in folder `hand_gestures` in this practicum) for each variable `df_rock`, `df_paper`, `df_scissors`, and `df_ok` using 

In [10]:
df_rock = pd.read_csv('./hand_gestures/0.csv', header = None)
df_paper = pd.read_csv('./hand_gestures/1.csv', header = None)
df_scissors = pd.read_csv('./hand_gestures/2.csv', header = None)
df_ok = pd.read_csv('./hand_gestures/3.csv', header = None)

# YOUR CODE HERE
#raise NotImplementedError()

df = pd.concat([df_rock, df_scissors, df_ok,  df_paper])

df.tail()
#df_ok

,0,1,2,3,4,5,6,7,8,9,...,55,56,57,58,59,60,61,62,63,64
2898,0.0,1.0,-2.0,-2.0,-11.0,-12.0,-2.0,-9.0,-7.0,-3.0,...,-7.0,-6.0,-1.0,0.0,1.0,38.0,-8.0,0.0,4.0,1
2899,-4.0,-4.0,1.0,-1.0,-52.0,-4.0,-1.0,-2.0,9.0,-1.0,...,0.0,-4.0,3.0,1.0,8.0,17.0,0.0,0.0,-7.0,1
2900,26.0,2.0,0.0,-1.0,20.0,-1.0,1.0,-4.0,18.0,-1.0,...,-7.0,10.0,0.0,-3.0,2.0,34.0,42.0,3.0,6.0,1
2901,-3.0,4.0,7.0,4.0,-26.0,-32.0,-3.0,-2.0,5.0,1.0,...,1.0,-1.0,0.0,1.0,-1.0,4.0,1.0,0.0,-5.0,1
2902,-16.0,-3.0,-8.0,-10.0,-24.0,-10.0,-4.0,-9.0,24.0,3.0,...,-3.0,-4.0,1.0,-1.0,-6.0,-21.0,-38.0,-4.0,8.0,1


In [ ]:
assert df.shape[0] == 11678
assert df.shape[1] == 65

Now convert the pandas dataframe to numpy matrices `X_4class` and `y_4class` for the features and labels respectively.

In [12]:
# hint: use df.iloc.values and Boolean indexing 

# YOUR CODE HERE
X_4class = df.iloc[:,0:65]
Y_4class = df.iloc[:,65]

#raise NotImplementedError()


IndexError: single positional indexer is out-of-bounds

In [ ]:
assert X_4class.shape[0] == 11678
assert y_4class.shape[0] == 11678
assert y_4class.ndim == 1

In [ ]:
#if you are stuck here, uncomment the lines below to load variables

npzfile = np.load('./get_unstuck/4class.npz')
X_4class = npzfile['X_4class']
y_4class = npzfile['y_4class']

Let's split the dataset into a training and testing set and train a random forest classifier. Use `random_state=0` as an argument for the classifier to ensure reproducible results.

In [ ]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_4class_train, X_4class_test, y_4class_train, y_4class_test = train_test_split(X_4class, y_4class, test_size = 0.25, random_state = 0)

# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc_4class = StandardScaler()
X_4class_train = sc_4class.fit_transform(X_4class_train)
X_4class_test = sc_4class.transform(X_4class_test)

# Defining and fitting classifier
# YOUR CODE HERE
#raise NotImplementedError()
rf_clf_4class.fit(X_4class_train, y_4class_train)

# Predicting the Test set results
rf_y_4class_pred = rf_clf_4class.predict(X_4class_test)
print_accuracy_confusion(y_4class_test, rf_y_4class_pred)

In [2]:
assert np.abs(accuracy_score(y_4class_test, rf_y_4class_pred)-0.9154109589041096)<1e-6

NameError: name 'np' is not defined

We are now are going to consider only 2 out of the 4 classes in this dataset, so we can focus on binary classifiers.
Based on the confusion matrix above, which classes are easiest to distinguish?

In [ ]:
class_1=42 # 42 is obviously nonsense, replace by the correct class
class_2=42 # ensure that class_1 < class_2

# YOUR CODE HERE
raise NotImplementedError()

Now we create 2 new variables `X_binary` and `y_binary` that only contain these 2 classes (the easiest to distinguish). Hint: use Boolean indexing and bitwise logic operators.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
assert X_binary.shape[0] == 5813
assert X_binary.shape[1] == 64
assert y_binary.ndim == 1
assert X_binary[0,0] == 26.0
assert X_binary[5812,0] == -16.0

In [ ]:
#if you are stuck here, uncomment the lines below to load variables

#npzfile = np.load('./get_unstuck/binary.npz')
#X_binary = npzfile['X_binary']
#y_binary = npzfile['y_binary']

Preparing the modified dataset...

In [ ]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_binary_train, X_binary_test, y_binary_train, y_binary_test = train_test_split(X_binary, y_binary, test_size = 0.25, random_state = 0)

# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc_binary = StandardScaler()
X_binary_train = sc_binary.fit_transform(X_binary_train)
X_binary_test = sc_binary.transform(X_binary_test)

Now let's create a few more classifiers to see how they perform.
- a SGD classifier `sgd_clf_binary` (with `loss='log', random_state=0`, all other parameters as default)
- a decision tree classifier `dt_clf_binary` (with `random_state=0`, all other parameters as default)
- a random forest classifier `rf_clf_binary` (with `random_state=0`, all other parameters as default)

define them, fit them, and predict the values for the test set (use the variable names in the stub that displays the results)

In [ ]:
# Defining and fitting mutliple classifiers

# YOUR CODE HERE
raise NotImplementedError()

# Printing results
print_accuracy_confusion(y_binary_test, sgd_y_binary_pred, 'sgd')
print_accuracy_confusion(y_binary_test, dt_y_binary_pred, 'dt')
print_accuracy_confusion(y_binary_test, rf_y_binary_pred, 'rf')

In [ ]:
assert np.abs(accuracy_score(y_binary_test, sgd_y_binary_pred)-0.5845942228335625)<1e-6
assert np.abs(accuracy_score(y_binary_test, dt_y_binary_pred)-0.9773039889958735)<1e-6
assert np.abs(accuracy_score(y_binary_test, rf_y_binary_pred)-0.9917469050894085)<1e-6

The dataset has 64 features, which are virtually impossible to visualize in 2D. The next steps are going to take us from those 64 features down to 2 features.

Take a look again at the description of the data set. Each sample contains measurements from 8 sensors, over 8 consecutive time steps. The measurements are very noisy anyhow, so average over the 8 time steps and create a new variable `X_binary_avg`. That can be done in 2 lines with numpy functions.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
assert X_binary_avg.shape[0] == 5813
assert X_binary_avg.shape[1] == 8
assert X_binary_avg[0,0] == 8
assert X_binary_avg[5812,0] == 1.375

In [ ]:
#if you are stuck here, uncomment the lines below to load variables

#npzfile = np.load('./get_unstuck/binary_avg.npz')
#X_binary_avg= npzfile['X_binary_avg']

Explore the dataset to see which features clearly distinguish the 2 classes. The function below can help with that.

In [ ]:
def plot_data(X, y, feat_1, feat_2, class_1=0, class_2=1):
    plt.plot(X[y==class_1,feat_1], X[y==class_1,feat_2], '.', label='class '+str(class_1))
    plt.plot(X[y==class_2,feat_1], X[y==class_2,feat_2], '.', label='class '+str(class_2))
    plt.grid('on')
    plt.axis('equal')
    plt.xlabel('feature '+str(feat_1))
    plt.ylabel('feature '+str(feat_2))
    plt.legend()

In [ ]:
# an empty cell for your own code

In [ ]:
feat_1 = 42 #42 is obviously nonsense, replace by the correct feature
feat_2 = 42 # ensure that feat_1 < feat_2
# YOUR CODE HERE
raise NotImplementedError()
plot_data(X_binary_avg, y_binary, feat_1, feat_2)

We can also use an auto-generated decision tree to help us decide on the must informative features.

In [ ]:
X_simple_train, X_simple_test, y_simple_train, y_simple_test = train_test_split(X_binary_avg, y_binary, test_size = 0.25, random_state = 0)

from sklearn.tree import DecisionTreeClassifier
dt_clf_simple = DecisionTreeClassifier(random_state=0, max_depth=3)
dt_clf_simple.fit(X_simple_train, y_simple_train)
dt_y_simple_pred = dt_clf_simple.predict(X_simple_test)

print_accuracy_confusion(y_simple_test, dt_y_simple_pred, 'dt_simple')

In [ ]:
from sklearn import tree
import graphviz 
dot_data = tree.export_graphviz(dt_clf_simple, out_file=None, filled=True, rounded=True)  
graph = graphviz.Source(dot_data) 
graph

Hopefully you'll come to a similar conclusion than with manual inspection.

In [ ]:
feat_1 = 42 #42 is obviously nonsense, replace by the correct feature
feat_2 = 42 # ensure that feat_1 < feat_2
# YOUR CODE HERE
raise NotImplementedError()

One of the features is clearly dominant. Which other feature to use is a bit less clear cut. In any case use the one with the lower number. We are next going to look at SVMs. Those work best with small to medium data sets.

Create a new subset of the data (`X_binary_subset` and `y_binary_subset`) containing only the 2 features you chose and only every 3rd sample.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
assert X_binary_subset.shape[0] == 1938
assert y_binary_subset.shape[0] == 1938
assert X_binary_subset.shape[1] == 2
assert X_binary_subset[0,0] == 0.5
assert X_binary_subset[1937,0] == -0.375
assert y_binary_subset[1937] == 1

In [ ]:
#if you are stuck here, uncomment the lines below to load variables

#npzfile = np.load('./get_unstuck/binary_subset.npz')
#X_binary_subset = npzfile['X_binary_subset']
#y_binary_subset = npzfile['y_binary_subset']

Again some setup of the data etc.

In [ ]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_subset_train, X_subset_test, y_subset_train, y_subset_test = train_test_split(X_binary_subset, y_binary_subset, test_size = 0.25, random_state = 0)

axes_subset = np.array([X_binary_subset[:, 0].min() - 1, X_binary_subset[:, 0].max() + 1, X_binary_subset[:, 1].min() - 1, X_binary_subset[:, 1].max() + 1])


Here is a random forest classifier to see how well we can still perform on this very reduced dataset.

In [ ]:
# Defining and fitting mutliple classifiers
from sklearn.ensemble import RandomForestClassifier

rf_clf_subset = RandomForestClassifier(random_state=0)
rf_clf_subset.fit(X_subset_train, y_subset_train)

# Predicting the Test set results
rf_y_subset_pred = rf_clf_subset.predict(X_subset_test)

print_accuracy_confusion(y_subset_test, rf_y_subset_pred, 'rf')
plot_feature_space_function(rf_clf_subset.predict, X_binary_subset, y_binary_subset, axes_subset)

We can still do surprisingly well compared to the full dataset, even though we have drastically reduced the number of features.

Now create an no-linear SVM classifier `svm_clf_subset` that beats the random forest classifier in terms of accuracy.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

print_accuracy_confusion(y_subset_test, svm_y_subset_pred, 'svm')
plot_feature_space_function(svm_clf_subset.predict, X_binary_subset, y_binary_subset, axes_subset)

In [ ]:
assert accuracy_score(y_subset_test, svm_y_subset_pred) > 0.90103

Next we are going to re-implement a linear SVM. As the data above can clearly not be separated by a linear decision boundary, so we'll transform it once more. This obviously does not remove the overlap of the classes. Finally, the definition of an SVM expects labels/targets with -1 for one class and +1 for the other, so we also change the labels accordingly.

In [ ]:
X_lin = np.abs(X_binary_subset)
y_lin = 2.*y_binary_subset - 1.

plot_data(np.abs(X_lin), y_lin, 0, 1, -1, 1)
plt.title('Absolute value')

# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_lin, y_lin, test_size = 0.25, random_state = 0)

axes = np.array([X_lin[:, 0].min() - 1, X_lin[:, 0].max() + 1, X_lin[:, 1].min() - 1, X_lin[:, 1].max() + 1])

Let's see how well linear classifiers do now...

In [ ]:
from sklearn.linear_model import SGDClassifier
sgd_clf = SGDClassifier(random_state=0)
sgd_clf.fit(X_train, y_train);
sgd_y_pred = sgd_clf.predict(X_test)
print_accuracy_confusion(y_test, sgd_y_pred, 'sgd')
plot_feature_space_function(sgd_clf.predict, X_lin, y_lin, axes, -1 ,1)

Actually a lot better than on the initial dataset...

Now let's test with a linear SVM (using a linear kernel achieves the same as using LinearSVC, but this way of doing it gives us access to the support vectors). We'll compare our own implementation to this one later on.

In [ ]:
from sklearn.svm import SVC
lin_svm_clf = SVC(kernel="linear", C=2)
lin_svm_clf.fit(X_train, y_train)
lin_svm_y_pred = lin_svm_clf.predict(X_test)

print_accuracy_confusion(y_test, lin_svm_y_pred, 'lin_svm')
plot_feature_space_function(lin_svm_clf.predict, X_lin, y_lin, axes, -1, 1)
svs = lin_svm_clf.support_vectors_
plt.scatter(svs[:, 0], svs[:, 1], s=1, facecolors='#009900', label='support vectors')
plt.legend()

Is the data linearly separable?
Do we hence use a hard margin or a soft margin SVM?

In [ ]:
#uncomment the correct answers

#linearly_separable = 'yes'
#linearly_separable = 'no'

#SVM_margin = 'hard'
#SVM_margin = 'soft'

# YOUR CODE HERE
raise NotImplementedError()


### Re-implemeting a linear SVM classifier

We are using CVXOPT to solve the quadratic programming problem. According to https://cvxopt.org/userguide/coneprog.html#quadratic-programming the optimization problem needs to be in the following form

$$\begin{array}[t]{ll}\mbox{minimize} & (1/2) x^TPx + q^T x \\\mbox{subject to} & Gx \preceq h \\ & Ax = b\end{array}$$

with the corresponding API
`cvxopt.solvers.qp(P, q[, G, h[, A, b[, solver[, initvals]]]])`

The dual form of the hard margin linear SVM objective is (page 169 of the book)

$$\min_{\alpha} \frac{1}{2} \sum_{i}^m \sum_{j}^m \alpha^{(i)}\alpha^{(j)}t^{(i)}t^{(j)}\mathbf{x}^{(i)\mathsf{T}}\mathbf{x}^{(j)}  - \sum_{i=1}^m \alpha^{(i)}$$
$$\textrm{subject to } \alpha^{(i)} \geq 0 $$

Rewriting the SVM objective in vector/matrix form, and changing the constraint to the correct form, we get

$$\min_{\alpha} \frac{1}{2} \boldsymbol{\alpha}^{\mathsf{T}} \mathbf{H} \boldsymbol{\alpha} -  \mathbf{1}^{\mathsf{T}}\boldsymbol{\alpha}$$
$$\textrm{subject to } -\boldsymbol{\alpha} \leq \boldsymbol{0} $$
where matrix $\mathbf{H}$ has elements $H_{i,j} = t^{(i)}t^{(j)}\mathbf{x}^{(i)\mathsf{T}}\mathbf{x}^{(j)}$

we additionally know that $\sum_i^m \alpha^{(i)} t^{(i)} = 0 $ (page 763) which can be rewritten to $\ \mathbf{t}^{\mathsf{T}}\boldsymbol{\alpha} = 0 $

Now map the terms from the SVM objective to the CVXOPT syntax

In [3]:
# use 'alpha', 'H', 't', 'ones', 'identity', zero', 'zeros'
# for negtive values '-alpha' etc.
x = '?'
P = '?'
q = '?'
G = '?'
h = '?'
A = '?'
b = '?'
# YOUR CODE HERE
raise NotImplementedError()

NotImplementedError: 

For the soft margin SVM the constraint changes to $C \geq \alpha^{(i)} \geq 0$ so we get
$$
\begin{aligned}
    \min_{\alpha} & \frac{1}{2} \boldsymbol{\alpha}^{\mathsf{T}} \mathbf{H} \boldsymbol{\alpha} -  \mathbf{1}^{\mathsf{T}}\boldsymbol{\alpha}
    \\
     \textrm{subject to } & -\boldsymbol{\alpha} \leq \boldsymbol{0} 
    \\
     & \boldsymbol{\alpha} \leq \boldsymbol{C}
     \\
     & \mathbf{t}^{\mathsf{T}}\boldsymbol{\alpha} = 0  
\end{aligned}$$
where the two inequalities can be stacked in a combined matrix/vector `G` and `h`.


The only remaining question is now: "How can we efficiently calculate $\mathbf{H}$?" Let's consider a simple example with 2 samples $\{x^{(1)}, x^{(2)}\} \in \mathbb{R}^2$ which are two dimensional vectors. i.e. $\mathbf{x}^{(1)} = (x_1^{(1)} , x_2^{(1)})^\mathsf{T}$

$$X = \begin{bmatrix} x_1^{(1)} & x_2^{(1)} \\ x_1^{(2)} & x_2^{(2)} \end{bmatrix} \ \ \ t = \begin{bmatrix} t^{(1)}  \\ t^{(2)} \end{bmatrix}$$

We now proceed to creating a new matrix $\mathbf{X}'$ where each input sample $\mathbf{x}$ is multiplied by the corresponding output label $t$. This can be done easily in Numpy using vectorization and padding.

$$\mathbf{X}' = \begin{bmatrix} x^{(1)}_1 t^{(1)} & x^{(1)}_2t^{(1)} \\
x^{(2)}_1t^{(2)} & x^{(2)}_2t^{(2)} \end{bmatrix}$$

Finally we take the matrix multiplication of $\mathbf{X}'$ and its transpose giving

$$\mathbf{H} = {\mathbf{X}'} {\mathbf{X}'^\mathsf{T}} = \begin{bmatrix} x^{(1)}_1 t^{(1)} & x^{(1)}_2t^{(1)} \\
x^{(2)}_1t^{(2)} & x^{(2)}_2t^{(2)} \end{bmatrix} \begin{bmatrix} x^{(1)}_1 t^{(1)} & x^{(2)}_1 t^{(2)}  \\
x^{(1)}_2t^{(1)} & x^{(2)}_2t^{(2)} \end{bmatrix}$$

$$\mathbf{H} = \begin{bmatrix}  x^{(1)}_1 x^{(1)}_1t^{(1)}t^{(1)} + x^{(1)}_2x^{(1)}_2t^{(1)}t^{(1)} & x^{(1)}_1 x^{(2)}_1t^{(1)}t^{(2)} + x^{(1)}_2x^{(2)}_2t^{(1)}t^{(2)} \\ x^{(2)}_1 x^{(1)}_1t^{(2)}t^{(1)} + x^{(2)}_2x^{(1)}_2t^{(2)}t^{(1)} & x^{(2)}_1 x^{(2)}_1t^{(2)}t^{(2)} + x^{(2)}_2x^{(2)}_2t^{(2)}t^{(2)} \end{bmatrix}$$

#### Install cvxopt
The `tf2` environment does not include a solver for QPs. So we need to install an additional package. This is easy to do on a command line with
`conda install cvxopt`

In [ ]:
#Importing with custom names to avoid issues with numpy / sympy matrix
from cvxopt import matrix as cvxopt_matrix
from cvxopt import solvers as cvxopt_solvers

#Initializing values
C = 2
m,n = X_train.shape

In [ ]:
#Calculating H
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
assert H.shape[0] == 1453
assert H.shape[1] == 1453
assert H[0,0] == 1.265625
assert H[-1,-1] == 0.5625

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
assert P.shape[0] == 1453
assert P.shape[1] == 1453

assert q.shape[0] == 1453
assert q.shape[1] == 1
assert q[0,0] == -1

assert G.shape[0] == 2906
assert G.shape[1] == 1453
assert G[-1,-1] == 1.0

assert h.shape[0] == 2906
assert h.ndim == 1
assert h[0] == 0.0

assert A.shape[0] == 1
assert A.shape[1] == 1453
assert A[0,0] == 1.0

assert b.shape[0] == 1
assert b.ndim == 1


In [ ]:
#if you are stuck here, uncomment the lines below to load variables

#npzfile = np.load('./get_unstuck/cvxopt.npz')
#P = npzfile['P']
#q = npzfile['q']
#G = npzfile['G']
#h = npzfile['h']
#A = npzfile['A']
#b = npzfile['b']

Now we are almost ready to run the solver...

In [ ]:
#Converting into cvxopt format
P = cvxopt_matrix(P)
q = cvxopt_matrix(q)
G = cvxopt_matrix(G)
h = cvxopt_matrix(h)
A = cvxopt_matrix(A)
b = cvxopt_matrix(b)

#Setting solver parameters 
cvxopt_solvers.options['show_progress'] = False
cvxopt_solvers.options['abstol'] = 1e-10
cvxopt_solvers.options['reltol'] = 1e-10
cvxopt_solvers.options['feastol'] = 1e-10

Run the solver and store the results in `sol`

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

alphas = np.array(sol['x'])

In [ ]:
assert isinstance(sol, dict)
assert 'primal objective' in sol.keys()
assert alphas.shape[0] == 1453


Calculate `w_svm` and `b_svm` (see page 169)

In [ ]:
#==================Computing parameters===============================#
# use a threshold of 1e-4 (rather than 0) for computing b

# YOUR CODE HERE
raise NotImplementedError()

#Display results
w_svm = w_svm.flatten()
#print('Alphas = \n',alphas[alphas > 1e-4])
print('w = ', w_svm)
print('b = ', b_svm)

In [ ]:
assert np.abs(w_svm[0] - -0.42666667) < 1e-6
assert np.abs(b_svm - 2.06127472524322) < 1e-6


Implement the SVC prediction (see page 165)

In [ ]:
def own_svm_predict(b,w,X):
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
assert own_svm_predict(1,[2, 3],[4, 5]) == 1
assert own_svm_predict(-1,[0, 1],[1, 0]) == -1

In [ ]:
own_svm_y_pred = own_svm_predict(b_svm,w_svm,X_test)
print_accuracy_confusion(y_test, own_svm_y_pred, 'own_svm')

# a few lines to make own_svm_predict work with plot_feature_space_function (without implementing a fullblown sklearn calssifier)
import functools
own_svm_predict_param = functools.partial(own_svm_predict, b_svm, w_svm)

plot_feature_space_function(own_svm_predict_param, X_lin, y_lin, axes, -1, 1)

Let's compare our solution to the one from sklearn

In [ ]:
print('w = ',lin_svm_clf.coef_)
print('b = ',lin_svm_clf.intercept_)
#print('Indices of support vectors = ', lin_svm_clf.support_)
#print('Support vectors = ', lin_svm_clf.support_vectors_)
#print('Number of support vectors for each class = ', lin_svm_clf.n_support_)
#print('Coefficients of the support vector in the decision function = \n', np.abs(lin_svm_clf.dual_coef_))

print_accuracy_confusion(y_test, lin_svm_y_pred, 'lin_svm')
plot_feature_space_function(lin_svm_clf.predict, X_lin, y_lin, axes, -1, 1)

Both the accuracy and the `w` parameter values should be close. Small differences occur depending on how the optimization is done (e.g., tolerances). As pointed out on page 763, there are multiple ways for calculating `b`. The book proposes to use the average rather than a single value. If we just use the first value instead, we are a lot closer to the sklearn parameter.

### Voting classifier

We already tried quite a few classifiers on this data set. Let's try to combine them in an ensemble.

First we'll define two more classifiers for the ensemble.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf_clf = RandomForestClassifier(random_state=0)
rf_clf.fit(X_train, y_train)
rf_y_pred = rf_clf.predict(X_test)
print_accuracy_confusion(y_test, rf_y_pred, 'rf')
plot_feature_space_function(rf_clf.predict, X_lin, y_lin, axes, -1, 1)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn_clf = KNeighborsClassifier(5)
knn_clf.fit(X_train, y_train)
knn_y_pred = knn_clf.predict(X_test)
print_accuracy_confusion(y_test, knn_y_pred, 'knn')
plot_feature_space_function(knn_clf.predict, X_lin, y_lin, axes, -1, 1)

Combine the predictions of the SGD classifier `sgd_clf` (we implemented before looking into how to implement a linear SVM), the random forest `rf_clf`, and of the KNN `knn_clf` in one variable `combined_pred`

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
assert combined_pred.shape[0] == 3
assert combined_pred.shape[1] == 485

Implement hard voting, i.e., generate a new variable `hard_voting_pred` where the prediction corresponds to that of the majority of the classifiers.

In [ ]:
# hint: use a `mode` function

# YOUR CODE HERE
raise NotImplementedError()

print_accuracy_confusion(y_test, hard_voting_pred, 'hard_voting')

In [ ]:
assert hard_voting_pred.shape[0] == 485
assert hard_voting_pred.shape[1] == 1 or hard_voting_pred.ndim == 1
assert accuracy_score(y_test, hard_voting_pred) > 0.8927

Even this very simple voting approach brings the 3 classifiers that don't perform that well individually to the same level as your best performing classifier we had so far, the linear SVM.

### Bagging
Now let's see what we can do with an ensemble of just our worst performing classifier, the SGD. The code is already provided below, your task is to adjust the parameters such that it beats the linear SVM by getting an accuracy >0.9

In [ ]:
# Bagging
from sklearn.ensemble import BaggingClassifier
from sklearn.linear_model import SGDClassifier

n_estimators=1
max_samples=1000
bootstrap=False

# YOUR CODE HERE
raise NotImplementedError()

bag_clf = BaggingClassifier(
    SGDClassifier(random_state=0), n_estimators=n_estimators,
    max_samples=max_samples, bootstrap=bootstrap, random_state=0)
bag_clf.fit(X_train, y_train)
bag_y_pred = bag_clf.predict(X_test)

print_accuracy_confusion(y_test, bag_y_pred, 'bag')

In [ ]:
assert accuracy_score(y_test, bag_y_pred) > 0.9

linear SVM implementation based on https://xavierbourretsicotte.github.io/SVM_implementation.html

## Imitation Learning and Motion Planning

Motion planning is a field of robotics in which robots have to plan and execute a trajectory in order to reach a goal. The following image shows a manipulator reaching a goal while avoiding obstacles.

<img src="img/manipulator_motion.png" alt="drawing" width="400"/>

Motion Planning algorithms have to be build/tunned by someone with knowledge in robotics/control. But, what if we aim to have, one day, robots in every type of setting, such as household environments? Would it be feasible to have a robotics/control engineer tunning robots for every case?

### Imitation Learning
A way to overcome this issue is to use Imitation Learning. Imitation Learning gives humans that are not experts in robotics/control the ability to program a robot. The idea is simple, a human provides a set of demontrations of how a task should be done and then the robot imitates this behavior using Supervised Learning.  

### Goal

In this part of the assignment, you will build a Motion Planning system that uses an Imitation Learning technique called Behavioral Cloning to fit a classifier to demonstrations provided by an oracle. 

* **Oracle:** a system that provides demonstrations of a task. In our motivation of the problem the oracle is a human. 

For practical reasons, in this assignment we are going to use a Motion Planning algorithm (Artificial Potential Fields, implementation from https://pythonrobotics.readthedocs.io/en/latest/modules/path_planning.html#potential-field-algorithm) as an oracle, and given that we are going to imitate its behavior, it is not required for you to know how this method works.

In [ ]:
from potential_field_planning import create_environment, potential_field_planning, classifier_planning
import numpy as np
import matplotlib.pyplot as plt

### First steps
1) We have imported the function `create_environment`, which randomly generates an environment that consists of a starting point [sx, sy], a goal [gx, gy], and a number of obstacles [ox, oy]. The dimension of ox and oy is equal to the number of obstacles. 

Let us create an environment with one obstacle:

In [ ]:
sx, sy, gx, gy, ox, oy = create_environment(n_obstacles=1, seed=1)

**IMPORTANT:** do not modify the seed values.

2) We also imported the function `potential_field_planning`, which is our oracle. This function takes as input the parameters of our environment and generates a collision-free path to the goal. The generated trajectory is saved in `trajectory_data`.

* If you change the flag `show` to False, the trajectory generation is not going to be displayed.
* The `delay` input controls the velocity in which the trajectory is generated when `show=True`.

Let us generate a trajectory using our oracle:

In [ ]:
%matplotlib qt
trajectory_data = potential_field_planning(sx, sy, gx, gy, ox, oy, show=True, delay=0.2)

* When `show=True` window will appear, be aware that this window could appear in the background of your display.

* The black star represents the starting point and the purple star the goal. The blue circle is the obstacle and the red dots are samples from the trajectory generated by the oracle. 

### Database

If we get trajectories from our oracle in multiple environments, we can build a database that we can later use to train a classifier.

#### Features
In the context of motion planning, the **feature vector** corresponds to the **state of the robot** at a specific moment in time in a specific environment.

**state vector** = [gxr, gyr, oxr1,...,oxrn, oyr1, ..., oyrn, ]

* **gxr**:  x distance from robot to goal
* **gyr**:  y distance from robot to goal
* **oxri**: x distance from robot to obstacle i
* **oyri**: y distance from robot to obstacle i

#### Classes
In this assingment, each **class** that our classifier can choose corresponds to an **action** that the robot can take. The options are, move: **north, east, south, west, north-east, north-west, south-east and south-west**. These classes are represented using a one-dimensional vector with integer values that go from 0 to 7. 

1) `trajectory_data` is a dictionary that contains the set of states visited by the robot and the actions taken by it in the environment. Each row of the state vector corresponds to the state of the robot in a different moment in time. Use the keys *'state'* and *'action'* to access to this data: 

In [ ]:
states = None
actions = None
# YOUR CODE HERE
raise NotImplementedError()
print('states:', state)
print('actions:', action)

In [ ]:
assert state[0, 1] == -4.661503021612904
assert state[4, 3] == -0.48529445878791044
assert state[10, 2] == -0.08430939979145613


As you can see, the dimension of our state vector is a function of the number of obstacles in our environment. 

2) Build a function that computes the state dimension as a function of n obstacles:

In [ ]:
def get_state_dim(n_obstacles):
    state_dim = None
    # YOUR CODE HERE
    raise NotImplementedError()
    return state_dim

In [ ]:
assert get_state_dim(2) == 6
assert get_state_dim(4) == 10
assert get_state_dim(7) == 16


3) Build a function that collects all of the visited states and actions from different environments in two arrays:
* states
* actions

`n_trajectories_database` corresponds to the number of different environments used to build the database.

In [ ]:
def generate_database(n_obstacles, n_trajectories_database, show=True, delay=0.01):
    # Initialize our arrays
    states = np.empty([0, get_state_dim(n_obstacles)])
    actions = np.empty([0])
    np.random.seed(20)  # reproducibility
    
    if show:
        %matplotlib qt
        
    for i in range(n_trajectories_database):            
        # Create environment
        sx, sy, gx, gy, ox, oy = None, None, None, None, None, None  # MODIFY
        # YOUR CODE HERE
        raise NotImplementedError()
        
        # Run planner and get a trajectory
        trajectory_data = None  # MODIFY
        # YOUR CODE HERE
        raise NotImplementedError()
        if show:
            plt.close()  # close plot

        # Append trajectory to arrays states and actions: ADD lines "states = YOUR ANSWER" and "actions = YOUR ANSWER"
        # YOUR CODE HERE
        raise NotImplementedError()
        
        # Print progress
        if (i + 1) % int(n_trajectories_database  / 5) == 0:
            print('Progress database: %d %%' % ((i + 1) / n_trajectories_database * 100))
    return states, actions

Create a database with 5 trajectories

In [ ]:
states, actions = generate_database(n_obstacles=1, n_trajectories_database=5, show=True, delay=0.1)

In [ ]:
assert states[0, 3] == -0.8858050663402328
assert states[12, 1] == 0.2358235070186181
assert states[30, 0] == 0.1973693755920074
assert actions[2] == 4.0
assert actions[15] == 0.0
assert actions[31] == 0.0


### Prepare data

Now that we have our database, we need to prepare the data for a classifier.

In [ ]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

Using the above imported functions, build the `data_preprocess` function.

In [ ]:
def data_preprocess(states, actions): 
    # Shuffle data with random_state=0
    shuffled_states, shuffled_actions = None, None  # MODIFY
    # YOUR CODE HERE
    raise NotImplementedError()

    # Separate data into train and test (10% of the data for test set with random_state=0)
    states_train, states_test, actions_train, actions_test = None, None, None, None  # MODIFY
    # YOUR CODE HERE
    raise NotImplementedError()
    
    # Scale data
    scaler = None  # MODIFY
        # ADD line that fits scaler to the training set
    states_train_scaled = None  # MODIFY
    states_test_scaled = None  # MODIFY
    # YOUR CODE HERE
    raise NotImplementedError()
    return states_train_scaled, states_test_scaled, actions_train, actions_test, scaler

Preprocess database:

In [ ]:
#if you are stuck here, uncomment the lines below to load variables

#npzfile = np.load('./get_unstuck/states_actions.npz')
#states = npzfile['states']
#actions= npzfile['actions']

In [ ]:
states_train_scaled, states_test_scaled, actions_train, actions_test, scaler = data_preprocess(states, actions)

In [4]:
assert states_train_scaled[10, 3] == 0.9453513019567489
assert states_test_scaled[2, 2] == -0.5490661199704467
assert actions_train[8] == 1.0
assert actions_test[5] == 4.0


NameError: name 'states_train_scaled' is not defined

### Train classifier with demonstrations
Build function that trains a classifier and gets its accuracy score and confusion matrix.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

In [ ]:
def train_classifier(states, actions):
    # Prepare data
    states_train_scaled, states_test_scaled, actions_train, actions_test, scaler = None, None, None, None, None  # MODIFY
    # YOUR CODE HERE
    raise NotImplementedError()
    
    # Train Random Forest with random_state=0
    clf = None  # MODIFY
        # ADD classifier fit
    # YOUR CODE HERE
    raise NotImplementedError()

    # Predict using test data
    actions_test_pred = None  # MODIFY
    # YOUR CODE HERE
    raise NotImplementedError()

    # Get accuracy score
    accuracy = None  # MODIFY

    # Plot confusion matrix
    conf_matrix = None  # MODIFY
    # YOUR CODE HERE
    raise NotImplementedError()
    print(f'Accuracy: {accuracy:.3f}')
    print('\nConfusion matrix:\n\n', conf_matrix)
    
    return clf, accuracy, conf_matrix, scaler

Train classifier

In [ ]:
clf, accuracy, conf_matrix, scaler = train_classifier(states, actions)

In [ ]:
assert accuracy == 1.000
assert conf_matrix[0, 0] == 3
assert conf_matrix[2, 2] == 2


### Use classifer to generate trajectories
Our classifier got an accuracy of 1.0, let's try to use it as a motion planner! 

The following function generates trajectories using our trained classifier and return the success rate of our planner. 

* A trajectory is considered to be successful if it reaches the goal before a time limit and does not collide with any obstacle.

* The `classifier_planning` function was imported at the beginning of the file and it is in charge of creating a path using a classifier in an environment. If *show=True*, the plot will show the classifier's trajectories in purple and the Artificial Potential Field planner trajectories in red.

In [ ]:
def generate_trajectories_classifier(clf, scaler, n_obstacles, n_trajectories_test, show, delay=0.01):
    success_counter = 0  # count the number of times the robot reached the goal
    np.random.seed(72)  # reproducibility 

    for i in range(n_trajectories_test):
        # Create environment
        sx, sy, gx, gy, ox, oy = create_environment(n_obstacles=n_obstacles, seed=np.random.randint(low=0, high=1e6))
       
        # Generate trajectory
        %matplotlib qt
        success = classifier_planning(sx, sy, gx, gy, ox, oy, n_obstacles, clf, scaler, delay=delay, show=show)
        plt.close()  # close plot

        # Count goal reached
        if success:
            success_counter += 1
            
        # Print if classifier fails
        if not success:
            print('Trial %i failed!' % i)
            
        # Print progress
        if (i + 1) % int(n_trajectories_test  / 5) == 0:
            print('Progress trials: %d %%' % ((i + 1) / n_trajectories_test * 100))
    
    success_rate = success_counter / n_trajectories_test
    print('\nSuccess rate: %.3f' % success_rate)
    return success_rate

Carefully read this function.

**Q1:** What is the variable `n_trajectories_test`?

**Q2:** Why does the `classifier_planning` function needs the *scaler* in one of its inputs?

**A1:**
    
**A2:**

YOUR ANSWER HERE

Run this function to get the success rate of our classifier over 50 environments and save it in the variable `suc_5t`.

*Tip:* if you want to get results fast, use `show=False`.

In [ ]:
#if you are stuck here, uncomment the lines below to load variables

# In this case it is required to have pickle installed: pip install pickle-mixin

#import pickle 
#with open('./get_unstuck/clf_fitted', 'rb') as config_dictionary_file:
#    config_dictionary = pickle.load(config_dictionary_file) 
    
#clf = config_dictionary['clf']
#scaler = config_dictionary['scaler']

In [ ]:
suc_5t = None  # MODIFY
# YOUR CODE HERE
raise NotImplementedError()

**Q:** The success rate of the classifier seems to be quite low, even though the accuracy obtained with the database is 1.0. Why?

**A:**

YOUR ANSWER HERE

### Repeat experiment in different setups
Build function that runs the experiment as a function of n_obstacles, n_trajectories_database and n_trajectories_test.

In [ ]:
def run_experiment(n_obstacles, n_trajectories_database, n_trajectories_test, show=True):
    # Create database
    print('1) Create database: \n')
    states, actions = None, None  # MODIFY
    # YOUR CODE HERE
    raise NotImplementedError()

    # Train classifier
    print('\n2) Train classifier: \n')
    clf, accuracy, conf_matrix, scaler = None, None, None, None  # MODIFY
    # YOUR CODE HERE
    raise NotImplementedError()

    # Generate trajectories with classifier
    print('\n3) Test classifier: \n')
    success_rate = None  # MODIFY
    # YOUR CODE HERE
    raise NotImplementedError()
    return success_rate

Repeat experiment with a database of 20 trajectories.

In [ ]:
suc_20t = run_experiment(n_obstacles=1, n_trajectories_database=20, n_trajectories_test=50, show=False)

**Q:** With a larger dataset the obtained accuracy decreased; however, the success rate increased. Explain.

**A:**
YOUR ANSWER HERE


Repeat experiment with a database of 100 trajectories.

In [ ]:
suc_100t = run_experiment(n_obstacles=1, n_trajectories_database=100, n_trajectories_test=50, show=False)

**Q:** There appears to be a trend. As the size of the database increases, also does the success rate of our planner. Conclude.

**A:**

YOUR ANSWER HERE

#### Let's make a plot
The following function collects the success rate of different setups in a list.

In [ ]:
def repeat_experiments(n_trajectories_list, n_obstacles):
    success_rate_list = []
    for i in n_trajectories_list:
        print('\nNumber trajectories database:', i)
        suc_it = run_experiment(n_obstacles=n_obstacles, n_trajectories_database=i, n_trajectories_test=50, show=False)
        success_rate_list.append(suc_it)
        
    return success_rate_list

Run this function to get the success rate of the classifier when trained with databases of different sizes.

*Warning: it may take some seconds to compute these results*

In [ ]:
n_trajectories_list = [5, 10, 20, 30, 40, 60, 80, 100, 200]  # Do not modify
success_rate_list = repeat_experiments(n_trajectories_list, n_obstacles=1)

Finally, let's plot these results

In [ ]:
#if you are stuck here, uncomment the lines below to load variables
#npzfile = np.load('./get_unstuck/plot1.npz')
#success_rate_list= npzfile['success_rate_list']

In [ ]:
n_trajectories_list = [5, 10, 20, 30, 40, 60, 80, 100, 200]  # Do not modify

%matplotlib inline
plt.plot(n_trajectories_list, success_rate_list)
plt.grid('on')
plt.xlabel('Number of trajectories in database')
plt.ylabel('Success rate')
plt.show()

**Q1:** The success rate quickly increases until around `n_trajectories_list=50`. Afterwards, an increasing trend can still be observed, but it is slower. Why do you think this is happening?

In real life, building this database would translate into human time. Human time is expensive. 

**Q2:** Would it be worth it to collect 200 trajectories. Explain.

**A1:**

**A2:**

YOUR ANSWER HERE

#### Repeat experiment in environments with 1, 2 and 3 obstacles

*Warning:* this may take several seconds.

In [ ]:
n_obstacles_list = [1, 2, 3]
n_trajectories_list = [5, 10, 20, 30, 40, 60, 80]  # Do not modify
success_rate_list_obstacle = []

for i in n_obstacles_list:
    print('\nNumber of obstacles:', i)
    success_rate_list = repeat_experiments(n_trajectories_list, n_obstacles=i)
    success_rate_list_obstacle.append(success_rate_list)

And plot these results

In [ ]:
#if you are stuck here, uncomment the lines below to load variables
#npzfile = np.load('./get_unstuck/plot2.npz')
#success_rate_list_obstacle= npzfile['success_rate_list_obstacle']

In [ ]:
n_trajectories_list = [5, 10, 20, 30, 40, 60, 80]  # Do not modify

%matplotlib inline
for i in range(len(n_obstacles_list)):
    plt.plot(n_trajectories_list, success_rate_list_obstacle[i], label=('%i obstacle' % (i+1)))
    
plt.grid('on')
plt.xlabel('Number of trajectories in database')
plt.ylabel('Success rate')
plt.legend()
plt.show()

**Q**: As the number of obstacles increases, what happens with the performance of the classifier? Explain.

*Hint:* Focus on the feature/state space.

**A:** 

YOUR ANSWER HERE